# Block Chain - Part 1

Using the tutorial : https://bigishdata.com/2017/10/17/write-your-own-blockchain-part-1-creating-storing-syncing-displaying-mining-and-proving-work/


## Let's build down up
Let's construct a block, and it's structure.
All it does is, given a dictionary, it acccepts(trusts) all it's data. It works like a dictionary

Q: How would create_self_hash() look

In [1]:
class Block(object):
    def __init__(self, dictionary):
        '''
        We're looking for index, timestamp, data, prev_hash, nonce
        '''
        for k, v in dictionary.items():
            setattr(self, k, v)
            if not hasattr(self, 'hash'): #in creating the first block, needs to be removed in future
                self.hash = self.create_self_hash()

    def __dict__(self):
        info = {}
        info['index'] = str(self.index)
        info['timestamp'] = str(self.timestamp)
        info['prev_hash'] = str(self.prev_hash)
        info['hash'] = str(self.hash)
        info['data'] = str(self.data)
        return info

    def __str__(self):
        return "Block<prev_hash: %s,hash: %s>" % (self.prev_hash, self.hash)
    
    def create_self_hash(self):
        return "hash placeholder"
    
    def self_save(self):
        return save_block(self)

Now that we have the basic structure, let's create the first block.
Notice that the block's prev_hash refers to None

In [2]:
import datetime

def create_first_block():
    block_data = {}
    block_data['index'] = 0
    block_data['timestamp'] = datetime.datetime.now()
    block_data['data'] = 'First block data'
    block_data['prev_hash'] = None
    block = Block(block_data)
    return block
new_block = create_first_block()

Let's set up the folder. Let's call it chaindata

In [3]:
import os,json

chaindata_dir = 'chaindata'

def save_block(block):
  chaindata_dir = 'chaindata'
  filename = '%s%s.json' % (chaindata_dir, block.index)
  with open(filename, 'w') as block_file:
    print(filename)
    print(new_block.__dict__())
    json.dump(block.__dict__(), block_file)

# make dir
if not os.path.exists(chaindata_dir):
    os.mkdir(chaindata_dir)
# if empty, create first block
if os.listdir(chaindata_dir) == []:
        first_block = create_first_block()
        first_block.self_save() #not implemented yet
save_block(new_block)

chaindata0.json
{'index': '0', 'timestamp': '2018-01-31 00:54:07.253190', 'prev_hash': 'None', 'hash': 'hash placeholder', 'data': 'First block data'}


## Sync locally - Step by step

* Before you start off, you need to sync to the nodes
* You can't locally, so we do some emulation
* Just read the blocks fromm local data.
* In the future, also ave to talk to peers to gather the blocks that were generated while we weren't running the node.

In [4]:
import json
def sync():
    node_blocks = []
    
    #We're assuming that the folder and at least the initial block exists
    chandata_dir = 'chaindata'
    
    if os.path.exists(chaindata_dir):
        
        for filename in os.listdir(chaindata_dir):
            if filename.endswith('json'):
                filepath = '%s/%s' % (chaindata_dir, filename)
                print(chaindata_dir,filepath)
                with open(filepath, 'r') as block_file:
                    block_info = json.load(block_file)
                    block_object = Block(block_info)
                    node_blocks.append(block_object)
    return node_blocks
peer_nodes = sync()
print(peer_nodes)

chaindata chaindata/1.json
chaindata chaindata/0.json
[<__main__.Block object at 0x7fa4b92d3128>, <__main__.Block object at 0x7fa4b92d3048>]


## Deploy the blockchain 
* Use Flask
* Do crazy deploy

In [5]:
from flask import Flask

node = Flask(__name__)
node_blocks = sync()

@node.route('/blockchain.json', methods=['GET'])

def blockchain():
    """
    Deplots our blockchain
    Lists of json of hashes of block info = 
    [index, timestamp, data, hash, prev_hash]
    """
    
    node_blocks = sync() 
    
    # Convert our blocks into dictionaries
    python_blocks = []
    
    for block in node_blocks:
        python_blocks.append(block.__dict__())
    json_blocks = json.dumps(python_bblocks)
    
    return json_blocks

if __name__ == '__main__':
    node.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


chaindata chaindata/1.json
chaindata chaindata/0.json


## Mining

* Gonna create header.
* Gonna header = index+prev_hash+data+timestamp

In [6]:
# Simple header. Bitcoin header is super complex as compared to this
def generate_header(index, prev_hash, data, timestamp, nonce):
    return str(index)+prev_hash+data+str(timestamp) + str(nonce)

def calculate_hash(index, prev_hash,data, timestamp, nonce):
    header_string = generate_header(index, prev_hash, data, timestamp, nonce)
    sha = hashlib.sha256()
    sha.update(header_string.encode('utf-8'))
    return sha.hexdigest()



### TO the actual mining

In [7]:
import hashlib
node_blocks = sync()
NUM_ZEROS = 4
def mine(last_block):
  index = int(last_block.index) + 1
  timestamp = datetime.datetime.now()
  print (timestamp)
  data = "I block #%s" % (int(last_block.index) + 1) #random string for now, not transactions
  prev_hash = last_block.hash
    
  nonce = 0
  block_hash = calculate_hash(index, prev_hash, data, timestamp, nonce)
  
  while str(block_hash[0:NUM_ZEROS]) != '0' * NUM_ZEROS:
    nonce += 1
    block_hash = calculate_hash(index, prev_hash, data, timestamp, nonce)
  block_data = {}
  block_data['index'] = int(last_block.index) + 1
  block_data['timestamp'] = datetime.datetime.now()
  block_data['data'] = "I block #%s" % last_block.index
  block_data['prev_hash'] = last_block.hash
  block_data['hash'] = block_hash
  return Block(block_data)

def save_block(block):
#   chaindata_dir = 'chaindata
  filename = '%s/%s.json' % (chaindata_dir, block.index)
  with open(filename, 'w') as block_file:
    print(new_block.__dict__())
    json.dump(block.__dict__(), block_file)

last_block = node_blocks[-1]
new_block = mine(last_block)
save_block(new_block)

chaindata chaindata/1.json
chaindata chaindata/0.json
2018-01-31 00:54:43.892556
{'index': '1', 'timestamp': '2018-01-31 00:54:44.064575', 'prev_hash': 'hash placeholder', 'hash': '00002a3a8f710246cd763b4106ac4c7c18f957b9e3bf369a0473f446d23b15bd', 'data': 'I block #0'}
